### 下载语料
这个压缩包里面存的是标题、正文部分。

In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles.xml.bz2

### 提取正文
将 xml 格式的 wiki 数据转换为 text 格式。

In [ ]:
!pip install gensim
from gensim.corpora import WikiCorpus

input = "./zhwiki-latest-pages-articles.xml.bz2"
output = './wiki.zh.text'
with open(output, 'w') as f:
    wiki =  WikiCorpus(input, lemmatize=False, dictionary={})
    for text in wiki.get_texts():
        str_line = ' '.join(text)
        f.write(str_line + '\n')

### 繁简转换
如果抽取中文的话需要将繁体转化为简体(维基百科的中文数据是繁简混杂的，里面包含大陆简体、台湾繁体、港澳繁体等多种不同的数据)。可以使用opencc进行转换，也可以使用其它繁简转换工具。
```
Options:
 -i [file], --input=[file]   Read original text from [file].
 -o [file], --output=[file]  Write converted text to [file].
 -c [file], --config=[file]  Load configuration of conversion from [file].
 -v, --version               Print version and build information.
 -h, --help                  Print this help.

```

In [ ]:
!opencc -i wiki.zh.text -o wiki.zh.text.jian -c zht2zhs.ini

### 编码转换
由于后续的分词需要使用utf-8格式的字符，而上述简体字中可能存在非utf-8的字符集，避免在分词时候进行到一半而出现错误，因此先进行字符格式转换。 

使用iconv命令将文件转换成utf-8编码。
```
 输入/输出格式规范：
  -f, --from-code=名称     原始文本编码
  -t, --to-code=名称       输出编码

 信息：
  -l, --list                 列举所有已知的字符集

 输出控制：
  -c                         从输出中忽略无效的字符
  -o, --output=文件        输出文件
  -s, --silent               关闭警告
      --verbose              打印进度信息

  -?, --help                 给出该系统求助列表
      --usage                给出简要的用法信息
  -V, --version              打印程序版本号

```

In [ ]:
!iconv -c -t UTF-8 wiki.zh.text.jian > wiki.zh.text.jian.utf8

### 分词处理

In [ ]:
!pip install jieba
!python -m jieba -d ' ' wiki.zh.text.jian.utf8 > wiki.zh.text.jian.utf8.seg

### word2vec 训练

In [ ]:
from gensim.models import word2vec
import logging

input = './wiki.zh.text.jian.utf8.seg'
output = './wiki.vector'
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.LineSentence(input)
model = word2vec.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
model.wv.save_word2vec_format(output, binary=False)

### word2vec 测试

In [1]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('./wiki.vector', binary=False)

In [2]:
# model['女人'] + model['国王'] - model['男人'] = model['皇后']
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7293198108673096),
 ('bride', 0.683803915977478),
 ('mistress', 0.6707652807235718),
 ('prince', 0.6648019552230835),
 ('wives', 0.6588137149810791),
 ('princess', 0.6529775857925415),
 ('queens', 0.6459839344024658),
 ('daughters', 0.6443694829940796),
 ('mother', 0.6377485990524292),
 ('godmother', 0.6289682388305664)]

In [3]:
model.similarity('woman', 'man')

0.6361447854063201

In [4]:
model.similarity('queen', 'king')

0.6681771014772736